# Data Wrangling

## Question 3: Data Wrangling - Duplicate Transactions

### You will notice a number of what look like duplicated transactions in the data set. One type of duplicated transaction is a reversed transaction, where a purchase is followed by a reversal. Another example is a multi-swipe, where a vendor accidentally charges a customer's card multiple times within a short time span.

### Can you programmatically identify reversed and multi-swipe transactions?

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
import matplotlib.pyplot as plt
import requests
import zipfile
import io
import os

In [2]:
# Load the JSON data into a DataFrame
filename = "transactions.txt"
if os.path.exists(filename):
    df = pd.read_json(filename, lines=True)
    df.replace("", pd.NA, inplace=True)
    print("Data successfully loaded into a DataFrame!")
else:
    print(f"The file {filename} does not exist.")

Data successfully loaded into a DataFrame!


In [3]:
# converting date columns to datetime
df.transactionDateTime = pd.to_datetime(df.transactionDateTime)
df.accountOpenDate = pd.to_datetime(df.accountOpenDate)
df.dateOfLastAddressChange = pd.to_datetime(df.dateOfLastAddressChange)
df.currentExpDate = pd.to_datetime(df.currentExpDate)

C:\Users\wangj\AppData\Local\Temp\ipykernel_32620\3427329.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.currentExpDate = pd.to_datetime(df.currentExpDate)


In [4]:
# Drop fully non columns
df.drop(['echoBuffer','merchantCity','merchantState','merchantZip','posOnPremises','recurringAuthInd'],axis=1,inplace=True) 

# Reversal transactions

In [5]:
card_data=df.copy() # make a copy

In [6]:
# Generate unique transaction key
def concat(x):
    y=str('T') + str(x)
    return y

In [7]:
card_data.reset_index(drop=False,inplace=True)
card_data['transactionKey']=card_data['index'].apply(lambda x: concat(x))
card_data.drop(['index'],axis=1,inplace=True)

In [8]:
card_data.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,transactionKey
0,737265056,737265056,5000,5000.0,2016-08-13 14:27:32,98.55,Uber,US,US,02,...,2015-03-14,414,414,1803,PURCHASE,0.0,False,False,False,T0
1,737265056,737265056,5000,5000.0,2016-10-11 05:05:54,74.51,AMC #191138,US,US,09,...,2015-03-14,486,486,767,PURCHASE,0.0,True,False,False,T1
2,737265056,737265056,5000,5000.0,2016-11-08 09:18:39,7.47,Play Store,US,US,09,...,2015-03-14,486,486,767,PURCHASE,0.0,False,False,False,T2
3,737265056,737265056,5000,5000.0,2016-12-10 02:14:50,7.47,Play Store,US,US,09,...,2015-03-14,486,486,767,PURCHASE,0.0,False,False,False,T3
4,830329091,830329091,5000,5000.0,2016-03-24 21:04:46,71.18,Tim Hortons #947751,US,US,02,...,2015-08-06,885,885,3143,PURCHASE,0.0,True,False,False,T4


In [9]:
# Select several columns that help to identify reversal transaction
card_data=card_data[['customerId','merchantName','transactionAmount','transactionType','transactionDateTime','transactionKey','isFraud']]

#Sort datasets based on several columns such as customerID, merchantName and transactionDateTime in ascending order.
card_data=card_data.sort_values(by=['customerId','merchantName','transactionDateTime'],ascending=True) 

In [10]:
# Filter transactions with transactionType 'REVERSAL'
reversal_transactions = card_data[card_data['transactionType'] == 'REVERSAL']
reversal_transactions

,customerId,merchantName,transactionAmount,transactionType,transactionDateTime,transactionKey,isFraud
541964,100088067,AMC #79863,22.32,REVERSAL,2016-11-20 08:00:04,T541964,False
87920,100328049,Lyft,43.74,REVERSAL,2016-01-15 20:36:18,T87920,False
87936,100328049,McDonalds #423357,284.97,REVERSAL,2016-03-26 17:35:09,T87936,False
151499,100737756,34th BBQ #436606,93.67,REVERSAL,2016-06-06 22:56:52,T151499,False
151541,100737756,Best Bistro #262998,501.29,REVERSAL,2016-08-24 20:21:19,T151541,False
...,...,...,...,...,...,...,...
545581,999258704,gap.com,72.93,REVERSAL,2016-09-02 03:33:07,T545581,False
670038,999283629,Delta Airlines,247.40,REVERSAL,2016-05-01 05:42:19,T670038,False
670109,999283629,Franks Sandwitch Bar,150.58,REVERSAL,2016-11-19 18:48:08,T670109,False
670052,999283629,Lyft,96.56,REVERSAL,2016-06-07 18:32:39,T670052,False


## I screened reversal transactions and original transactions based on the following criteria:

**1.Each reversal transaction must correspond to one original transaction.**

**2.The amount of a reversal transaction cannot exceed that of the original transaction.**

**3.The reversal transaction must occur within 30 days of the original transaction.**

**4.A pair of reversal and original transactions must belong to the same customerId and merchantName.**

**5. The reversal transaction is not necessarily adjacent to the original transaction.**

**6.Partial reversals may occur in certain cases.**


In [11]:
# Merge reversal transactions with the rest of the dataset on customerId and merchantName
merged_transactions = pd.merge(
    reversal_transactions[['customerId', 'merchantName', 'transactionDateTime', 'transactionAmount', 'transactionType','transactionKey','isFraud']],
    card_data,
    on=['customerId', 'merchantName'],
    suffixes=('_reversal', '_original')
)

In [12]:
# Filter for transactions meeting the conditions
filtered_transactions = merged_transactions[
    (merged_transactions['transactionDateTime_original'] < merged_transactions['transactionDateTime_reversal']) &
    (merged_transactions['transactionDateTime_original'] >= merged_transactions['transactionDateTime_reversal'] - pd.Timedelta(days=30)) &
    (merged_transactions['transactionAmount_original'] >= merged_transactions['transactionAmount_reversal']) &
    (merged_transactions['transactionType_original'] != 'REVERSAL' )
]

In [13]:
# Calculate the absolute difference in transaction amounts
filtered_transactions['amountAdjusted'] = (
    filtered_transactions['transactionAmount_original'] - filtered_transactions['transactionAmount_reversal']
).abs()

# Find the transaction with the smallest amount difference for each reversal
reversal_original_transactions = filtered_transactions.loc[
    filtered_transactions.groupby('transactionKey_reversal')['amountAdjusted'].idxmin()
]

C:\Users\wangj\AppData\Local\Temp\ipykernel_32620\980876421.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_transactions['amountAdjusted'] = (


In [14]:
# This df shows the reserval transactions and original transactions
reversal_original_transactions

,customerId,merchantName,transactionDateTime_reversal,transactionAmount_reversal,transactionType_reversal,transactionKey_reversal,isFraud_reversal,transactionAmount_original,transactionType_original,transactionDateTime_original,transactionKey_original,isFraud_original,amountAdjusted
4417877,740574175,ebay.com,2016-10-27 23:30:28,118.87,REVERSAL,T100015,False,118.87,PURCHASE,2016-10-27 23:29:02,T100014,False,0.00
4415700,740574175,Pizza Hut #713576,2016-11-11 21:20:25,27.64,REVERSAL,T100097,False,27.64,PURCHASE,2016-10-19 07:29:15,T99964,False,0.00
4415697,740574175,Pizza Hut #527039,2016-11-13 21:36:40,29.42,REVERSAL,T100107,False,29.42,PURCHASE,2016-11-13 21:36:14,T100106,False,0.00
4418767,740574175,staples.com,2016-11-20 06:31:21,6.07,REVERSAL,T100146,False,6.07,PURCHASE,2016-11-20 06:30:16,T100145,False,0.00
4416433,740574175,amazon.com,2016-11-21 08:01:32,19.29,REVERSAL,T100150,False,65.43,PURCHASE,2016-11-11 11:18:03,T100094,False,46.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419170,740574175,target.com,2016-09-30 14:14:17,147.75,REVERSAL,T99861,False,147.75,PURCHASE,2016-09-09 16:11:54,T99713,False,0.00
4417754,740574175,ebay.com,2016-09-30 22:19:22,28.05,REVERSAL,T99864,False,28.05,PURCHASE,2016-09-18 23:31:48,T99779,False,0.00
4415723,740574175,Wendys #760583,2016-10-07 20:11:11,84.17,REVERSAL,T99899,False,84.17,PURCHASE,2016-09-30 14:51:22,T99862,False,0.00
4418045,740574175,oldnavy.com,2016-10-09 19:08:11,18.32,REVERSAL,T99910,False,48.93,PURCHASE,2016-09-15 02:33:23,T99755,False,30.61


## Fully reversal transaction

In [15]:
# Select fully reversal transactions where transactionAmount_reversal is equal to transactionAmount_original
fully_reversal= reversal_original_transactions[reversal_original_transactions['amountAdjusted']==0]

In [16]:
fully_reversal

,customerId,merchantName,transactionDateTime_reversal,transactionAmount_reversal,transactionType_reversal,transactionKey_reversal,isFraud_reversal,transactionAmount_original,transactionType_original,transactionDateTime_original,transactionKey_original,isFraud_original,amountAdjusted
4417877,740574175,ebay.com,2016-10-27 23:30:28,118.87,REVERSAL,T100015,False,118.87,PURCHASE,2016-10-27 23:29:02,T100014,False,0.0
4415700,740574175,Pizza Hut #713576,2016-11-11 21:20:25,27.64,REVERSAL,T100097,False,27.64,PURCHASE,2016-10-19 07:29:15,T99964,False,0.0
4415697,740574175,Pizza Hut #527039,2016-11-13 21:36:40,29.42,REVERSAL,T100107,False,29.42,PURCHASE,2016-11-13 21:36:14,T100106,False,0.0
4418767,740574175,staples.com,2016-11-20 06:31:21,6.07,REVERSAL,T100146,False,6.07,PURCHASE,2016-11-20 06:30:16,T100145,False,0.0
4415676,740574175,Hardee's #43185,2016-11-25 14:01:37,176.83,REVERSAL,T100179,False,176.83,PURCHASE,2016-11-22 13:40:54,T100158,False,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4415693,740574175,KFC #632134,2016-09-30 08:42:28,34.39,REVERSAL,T99860,False,34.39,PURCHASE,2016-09-05 15:23:44,T99691,False,0.0
4419170,740574175,target.com,2016-09-30 14:14:17,147.75,REVERSAL,T99861,False,147.75,PURCHASE,2016-09-09 16:11:54,T99713,False,0.0
4417754,740574175,ebay.com,2016-09-30 22:19:22,28.05,REVERSAL,T99864,False,28.05,PURCHASE,2016-09-18 23:31:48,T99779,False,0.0
4415723,740574175,Wendys #760583,2016-10-07 20:11:11,84.17,REVERSAL,T99899,False,84.17,PURCHASE,2016-09-30 14:51:22,T99862,False,0.0


In [17]:
# fully reversal transaction contains a small amount of fraud
fully_reversal.describe(include=['bool']) 

,isFraud_reversal,isFraud_original
count,18338,18338
unique,2,2
top,False,False
freq,18028,18028


## Partial Reversal

In [18]:
# Select partial reversal transactions where transactionAmount_reversal is smaller than transactionAmount_original
partial_reversal =  reversal_original_transactions[reversal_original_transactions['amountAdjusted']!=0]

In [19]:
partial_reversal

,customerId,merchantName,transactionDateTime_reversal,transactionAmount_reversal,transactionType_reversal,transactionKey_reversal,isFraud_reversal,transactionAmount_original,transactionType_original,transactionDateTime_original,transactionKey_original,isFraud_original,amountAdjusted
4416433,740574175,amazon.com,2016-11-21 08:01:32,19.29,REVERSAL,T100150,False,65.43,PURCHASE,2016-11-11 11:18:03,T100094,False,46.14
4415673,740574175,Dunkin' Donuts #614833,2016-12-25 20:38:28,9.26,REVERSAL,T100367,False,45.63,PURCHASE,2016-11-28 02:02:35,T100200,False,36.37
201902,249676988,Rodeway Inn #704793,2016-02-17 14:20:13,143.84,REVERSAL,T100853,False,234.23,PURCHASE,2016-02-17 14:20:05,T100852,False,90.39
201909,249676988,Starbucks #454714,2016-07-14 00:30:09,3.54,REVERSAL,T101045,False,8.14,PURCHASE,2016-06-23 05:08:22,T101016,False,4.60
4155315,454175276,staples.com,2016-01-30 14:44:15,7.70,REVERSAL,T101987,False,13.75,PURCHASE,2016-01-18 10:00:58,T101908,False,6.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4415710,740574175,Subway #808905,2016-03-16 19:15:17,29.30,REVERSAL,T98653,False,68.60,PURCHASE,2016-03-16 19:13:39,T98652,False,39.30
4419329,740574175,walmart.com,2016-04-13 12:43:45,25.58,REVERSAL,T98815,False,30.07,PURCHASE,2016-04-13 12:42:28,T98814,False,4.49
4417289,740574175,discount.com,2016-06-27 18:10:48,20.35,REVERSAL,T99260,False,26.41,PURCHASE,2016-06-03 04:45:05,T99128,False,6.06
4416308,740574175,amazon.com,2016-09-24 03:01:30,80.51,REVERSAL,T99816,False,85.32,PURCHASE,2016-09-23 00:08:14,T99808,False,4.81


In [20]:
# partial reversal transaction contains a small amount of fraud
partial_reversal.describe(include=['bool']) 

,isFraud_reversal,isFraud_original
count,1965,1965
unique,2,2
top,False,False
freq,1938,1941


## What total number of transactions and total dollar amount do you estimate for the reversed transactions?

In [21]:
# Calculate the total number of reversed transactions
total_reversed_count = len(reversal_transactions)

# Calculate the total number of fully reversed transactions
fully_reversed_count = len(fully_reversal)

# Calculate the total number of partial reversed transactions
partial_reversed_count = len(partial_reversal)

# Calculate the total dollar amount for reversed transactions
total_reversed_amount = round(reversal_transactions['transactionAmount'].sum(),2)

# Calculate the total dollar amount for Total Original Amount of Reversed Transactions
total_original_amount = round(reversal_original_transactions['transactionAmount_original'].sum(),2)

# Calculate the reversed ratio as (Total Reversed Amount / Total Original Amount)
reversed_ratio = round(total_reversed_amount / total_original_amount,2)

# Calculate the average reversed amount
reversed_amount = round(total_reversed_amount / total_reversed_count,2)

# Calculate the fraud ratio
fraud_ratio_fully_reversal = fully_reversal['isFraud_reversal'].mean() * 100
fraud_ratio_partial_reversal = partial_reversal['isFraud_reversal'].mean() * 100

# Prepare the results
reversed_summary = {
    "Total Number of Reversed Transactions": total_reversed_count,
    "Total Number of Reversed Transactions": fully_reversed_count,
    "Total Number of Partial Transactions": partial_reversed_count,
    "Total Dollar Amount of Reversed Transactions": total_reversed_amount,
    "Total Original Amount of Reversed Transactions": total_original_amount,
    "Reversed Ratio": reversed_ratio,
    "Average Reversed Amount": reversed_amount,
    "Proportion of Reversed Transactions": round(total_reversed_count/len(df), 2),
    "Fraud Ratio in Fully Reversal Transactions": fraud_ratio_fully_reversal,
    "Fraud Ratio in Partial Reversal Transactions": fraud_ratio_partial_reversal 
}

print(reversed_summary)

{'Total Number of Reversed Transactions': 18338, 'Total Number of Partial Transactions': 1965, 'Total Dollar Amount of Reversed Transactions': 2821792.5, 'Total Original Amount of Reversed Transactions': 2930125.06, 'Reversed Ratio': 0.96, 'Average Reversed Amount': 138.98, 'Proportion of Reversed Transactions': 0.03, 'Fraud Ratio in Fully Reversal Transactions': 1.690478787217799, 'Fraud Ratio in Partial Reversal Transactions': 1.3740458015267176}


## Did you find anything interesting about either kind of transaction?

Out of a total of 18,338 reversal transactions, only 1,965 are partial reversals, which means the majority of reversals (almost 90%) are full reversals.

Partial reversals account for about 10.7% of all reversal transactions, indicating they are less common but still notable. This might represent situations like refunds for a portion of the transaction (e.g., returned items).

The reversed ratio of 0.96 suggests that most reversal amounts are close to their original transaction amounts, reinforcing that partial reversals are relatively rare.

The average reversed transaction is $138.98, consistent with mid-sized transactions being the primary target for reversals.

Reversed transactions constitute only 3% of all transactions, indicating they are relatively rare. 

Fully reversal transactions and partial reversal transactions contains 1% of fraud


# Multi-swipe transactions

### I screened Multi-swipe transactions based on the following assumptions:

**1.Multi-swipe transactions occur in the PURCHASE category and not in Reversal or Address Check categories.**

**2. Multi-swipe transactions involve a customer making two or more payments to the same merchant.**

**3. Multi-swipe transactions occur when a customer makes consecutive payments to the same merchant with a short time interval between two transactions, assumed to be 10 minutes.**

In [22]:
purchase_transactions = card_data[card_data['transactionType'] == 'PURCHASE']

In [23]:
# Sort the dataset by customerId, merchantName, and transactionDateTime
purchase_transactions = purchase_transactions.sort_values(by=['customerId', 'merchantName', 'transactionDateTime'])

# Calculate the delta (difference) in transactionDateTime for consecutive transactions
purchase_transactions['deltaTransactionTime'] = purchase_transactions.groupby(
    ['customerId', 'merchantName']
)['transactionDateTime'].diff()

In [24]:
# Group by customerId, merchantName, and transactionAmount, and count the occurrences
amount_counts = purchase_transactions.groupby(['customerId', 'merchantName', 'transactionAmount']).size().reset_index(name='AmountCount')

# Merge the counts back into the original purchase_transactions DataFrame
purchase_transactions = purchase_transactions.merge(
    amount_counts, 
    on=['customerId', 'merchantName', 'transactionAmount'], 
    how='left'
)

In [25]:
# Filter transactions where buyer paid more than once to the same business
filtered_transactions = purchase_transactions[purchase_transactions['AmountCount'] > 1]

In [26]:
filtered_transactions

,customerId,merchantName,transactionAmount,transactionType,transactionDateTime,transactionKey,isFraud,deltaTransactionTime,AmountCount
13,100088067,Apple iTunes,3.96,PURCHASE,2016-05-05 22:47:50,T541919,False,NaT,8
14,100088067,Apple iTunes,3.96,PURCHASE,2016-06-05 22:10:41,T541926,False,30 days 23:22:51,8
15,100088067,Apple iTunes,3.96,PURCHASE,2016-07-05 22:22:49,T541929,False,30 days 00:12:08,8
16,100088067,Apple iTunes,3.96,PURCHASE,2016-08-05 00:55:48,T541936,False,30 days 02:32:59,8
17,100088067,Apple iTunes,3.96,PURCHASE,2016-09-05 13:14:22,T541945,False,31 days 12:18:34,8
...,...,...,...,...,...,...,...,...,...
745186,999389635,Play Store,4.62,PURCHASE,2016-08-22 11:42:48,T108110,False,31 days 10:14:38,6
745187,999389635,Play Store,4.62,PURCHASE,2016-09-22 13:58:30,T108112,False,31 days 02:15:42,6
745188,999389635,Play Store,4.62,PURCHASE,2016-10-23 16:47:41,T108113,False,31 days 02:49:11,6
745189,999389635,Play Store,4.62,PURCHASE,2016-11-24 00:21:46,T108114,False,31 days 07:34:05,6


In [27]:
# Reset the index of the original filtered_transactions DataFrame
filtered_transactions = filtered_transactions.reset_index(drop=True)

# Filter rows where deltaTransactionTime is less than 10 minutes
multiswipes = filtered_transactions[filtered_transactions['deltaTransactionTime'] < pd.Timedelta(minutes=10)]

# Calculate the valid indices for the previous rows
previous_indices = multiswipes.index[multiswipes.index > 0] - 1

# Combine the multiswipes with the valid previous rows
total_filtered_rows = pd.concat([
    filtered_transactions.loc[multiswipes.index],
    filtered_transactions.loc[previous_indices]
]).drop_duplicates()

# Sort the filtered rows by customerId and merchantName
total_filtered_rows = total_filtered_rows.sort_values(by=['customerId', 'merchantName','transactionDateTime']).reset_index(drop=True)

In [28]:
# multiswipe transactions and original transactions
total_filtered_rows

,customerId,merchantName,transactionAmount,transactionType,transactionDateTime,transactionKey,isFraud,deltaTransactionTime,AmountCount
0,100088067,Fresh Flowers,411.35,PURCHASE,2016-10-16 18:01:00,T541955,False,44 days 14:20:06,2
1,100088067,Fresh Flowers,411.35,PURCHASE,2016-10-16 18:01:02,T541956,False,0 days 00:00:02,2
2,100737756,34th BBQ #166379,43.25,PURCHASE,2016-07-10 14:31:07,T151517,False,138 days 00:20:01,2
3,100737756,34th BBQ #166379,43.25,PURCHASE,2016-07-10 14:32:06,T151518,False,0 days 00:00:59,2
4,100737756,Franks Deli,693.50,PURCHASE,2016-01-18 01:55:24,T151426,False,NaT,3
...,...,...,...,...,...,...,...,...,...
14711,999086814,staples.com,172.50,PURCHASE,2016-08-04 00:04:03,T388759,False,0 days 00:01:36,2
14712,999257059,Wall Street News,47.58,PURCHASE,2016-01-14 16:48:41,T569653,False,NaT,2
14713,999257059,Wall Street News,47.58,PURCHASE,2016-01-14 16:51:22,T569654,False,0 days 00:02:41,2
14714,999283629,Delta Airlines,247.40,PURCHASE,2016-04-09 12:56:43,T670026,False,NaT,2


In [29]:
# Sort the filtered rows by customerId and merchantName
multiswipes = multiswipes.sort_values(by=['customerId', 'merchantName','transactionDateTime']).reset_index(drop=True)

In [30]:
multiswipes

,customerId,merchantName,transactionAmount,transactionType,transactionDateTime,transactionKey,isFraud,deltaTransactionTime,AmountCount
0,100088067,Fresh Flowers,411.35,PURCHASE,2016-10-16 18:01:02,T541956,False,0 days 00:00:02,2
1,100737756,34th BBQ #166379,43.25,PURCHASE,2016-07-10 14:32:06,T151518,False,0 days 00:00:59,2
2,100737756,Franks Deli,693.50,PURCHASE,2016-01-18 01:55:28,T151427,False,0 days 00:00:04,3
3,100737756,Franks Deli,693.50,PURCHASE,2016-01-18 01:58:26,T151428,False,0 days 00:02:58,3
4,100737756,South Steakhouse #73819,211.22,PURCHASE,2016-07-02 12:07:00,T151513,False,0 days 00:01:56,2
...,...,...,...,...,...,...,...,...,...
7727,999086814,apple.com,22.76,PURCHASE,2016-12-19 02:49:22,T389018,False,0 days 00:00:58,2
7728,999086814,sears.com,178.57,PURCHASE,2016-01-23 09:22:35,T388400,False,0 days 00:02:33,2
7729,999086814,staples.com,172.50,PURCHASE,2016-08-04 00:04:03,T388759,False,0 days 00:01:36,2
7730,999257059,Wall Street News,47.58,PURCHASE,2016-01-14 16:51:22,T569654,False,0 days 00:02:41,2


In [31]:
# Calculate the total number of rows in the filtered dataset
total_filtered_rows = len(multiswipes)

# Calculate the total transactionAmount for the filtered dataset
total_transaction_amount = multiswipes['transactionAmount'].sum()

# Calculate the average transaction amount for the filtered dataset
average_transaction_amount = total_transaction_amount / total_filtered_rows if total_filtered_rows > 0 else 0

# Calculate the fraud ratio
fraud_ratio = multiswipes['isFraud'].mean() * 100

# Prepare the results
summary = {
    "Total Number of Multi-swipse Transactions": total_filtered_rows,
    "Total Transaction of Multi-swipse Amount": round(total_transaction_amount, 2),
    "Average Transaction Amount for Multi-swipse": round(average_transaction_amount, 2),
    "Proportion of Multi-swipse Transactions": round(total_filtered_rows/len(df), 2),
    "Multiswipes Fraud Ratio":  fraud_ratio

}

# Display the summary
summary


{'Total Number of Multi-swipse Transactions': 7732,
 'Total Transaction of Multi-swipse Amount': 1126124.69,
 'Average Transaction Amount for Multi-swipse': 145.64,
 'Proportion of Multi-swipse Transactions': 0.01,
 'Multiswipes Fraud Ratio': 1.7459906880496638}

## Did you find anything interesting about either kind of transaction?

Multi-swipe transactions account for approximately 1% of the total transactions, indicating they are relatively rare. 

The average transaction amount is $145.63, indicating that multi-swipe transactions mostly occur in moderate average transaction amount.

Multi swipe transactions contains a small proportion of fraud cases

# Remove Duplicates

### 1. Remove all fully reversed transactions.

### 2. Remove all multi-swipe transactions.

In [32]:
# Combine the three lists into one, removing duplicates
combined_keys = pd.concat([
    reversal_original_transactions['transactionKey_reversal'],
    fully_reversal['transactionKey_original'],
    multiswipes['transactionKey']
])

# Define the condition for filtering
filtered_condition = ~(
    (card_data['transactionKey'].isin(combined_keys) )
)

# Filter the DataFrame based on the new condition
credit_card_no_duplicate = card_data[filtered_condition]

### 3. Adjust the transaction amount of original transactions corresponding to partial reversal transactions.

In [33]:
# Count duplicates in the combined list
duplicate_count = combined_keys.duplicated().sum()

# Prepare the results
duplicate_summary = {
    "Total Keys Combined": len(combined_keys),
    "Duplicate Keys": duplicate_count,
    "Unique Keys": len(combined_keys) - duplicate_count
}

duplicate_summary

{'Total Keys Combined': 46373, 'Duplicate Keys': 11, 'Unique Keys': 46362}

In [34]:
credit_card_no_duplicate

,customerId,merchantName,transactionAmount,transactionType,transactionDateTime,transactionKey,isFraud
541917,100088067,1st BBQ,200.12,PURCHASE,2016-04-08 19:56:36,T541917,False
541962,100088067,1st Pub,95.11,PURCHASE,2016-11-09 07:26:06,T541962,False
541920,100088067,1st Sandwitch Bar #758805,5.26,PURCHASE,2016-05-06 02:54:01,T541920,False
541904,100088067,34th BBQ #412895,141.60,PURCHASE,2016-02-03 04:48:54,T541904,False
541925,100088067,AMC #191138,304.15,PURCHASE,2016-06-03 18:50:55,T541925,False
...,...,...,...,...,...,...,...
108113,999389635,Play Store,4.62,PURCHASE,2016-10-23 16:47:41,T108113,False
108114,999389635,Play Store,4.62,PURCHASE,2016-11-24 00:21:46,T108114,False
108115,999389635,Play Store,4.62,PURCHASE,2016-12-24 05:19:30,T108115,False
108109,999389635,oldnavy.com,42.98,PURCHASE,2016-08-04 20:27:36,T108109,False


In [35]:
# Create a mapping from transactionKey_original to amountDiff in partial_reversal
reversal_mapping = partial_reversal.set_index('transactionKey_original')['amountAdjusted'].to_dict()

# Replace transactionAmount in credit_card_no_duplicate using the mapping
credit_card_no_duplicate['transactionAmount'] = credit_card_no_duplicate.apply(
    lambda row: reversal_mapping[row['transactionKey']] 
    if row['transactionKey'] in reversal_mapping else row['transactionAmount'], 
    axis=1
)

C:\Users\wangj\AppData\Local\Temp\ipykernel_32620\2262566020.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  credit_card_no_duplicate['transactionAmount'] = credit_card_no_duplicate.apply(


In [36]:
credit_card_no_duplicate

,customerId,merchantName,transactionAmount,transactionType,transactionDateTime,transactionKey,isFraud
541917,100088067,1st BBQ,200.12,PURCHASE,2016-04-08 19:56:36,T541917,False
541962,100088067,1st Pub,95.11,PURCHASE,2016-11-09 07:26:06,T541962,False
541920,100088067,1st Sandwitch Bar #758805,5.26,PURCHASE,2016-05-06 02:54:01,T541920,False
541904,100088067,34th BBQ #412895,141.60,PURCHASE,2016-02-03 04:48:54,T541904,False
541925,100088067,AMC #191138,304.15,PURCHASE,2016-06-03 18:50:55,T541925,False
...,...,...,...,...,...,...,...
108113,999389635,Play Store,4.62,PURCHASE,2016-10-23 16:47:41,T108113,False
108114,999389635,Play Store,4.62,PURCHASE,2016-11-24 00:21:46,T108114,False
108115,999389635,Play Store,4.62,PURCHASE,2016-12-24 05:19:30,T108115,False
108109,999389635,oldnavy.com,42.98,PURCHASE,2016-08-04 20:27:36,T108109,False


# Save the processed data

In [37]:
df.reset_index(drop=False,inplace=True)
df['transactionKey']=df['index'].apply(lambda x: concat(x))
df.drop(['index'],axis=1,inplace=True)

In [38]:
# Assuming df and credit_card_no_duplicate are already loaded DataFrames
transaction_no_duplicate = df[df['transactionKey'].isin(credit_card_no_duplicate['transactionKey'])]
transaction_no_duplicate.drop(['transactionKey'],axis=1,inplace=True)
transaction_no_duplicate.to_parquet('transaction_no_duplicate.parquet', engine='pyarrow')

C:\Users\wangj\AppData\Local\Temp\ipykernel_32620\1951639226.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_no_duplicate.drop(['transactionKey'],axis=1,inplace=True)
